<a href="https://colab.research.google.com/github/medoyahya/assignment/blob/main/ali_emad_deep_learning_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
path_train='/content/drive/MyDrive/train'
path_test='/content/drive/MyDrive/test'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,MaxPooling2D
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
import glob as gb
from keras.applications.vgg16 import VGG16

In [7]:
code = {'Class (1)': 0, 'Class (20)': 1}

def getcode(n):
    for x, y in code.items():
        if n == y:
            return x

In [8]:
x_train = []
y_train = []
img_size = 220

for i in os.listdir(path_train):
    files = gb.glob(pathname=str(os.path.join(path_train, i, '*.jpg')))

    for j in tqdm(files):
        image = cv2.imread(j)
        image_array = cv2.resize(image, (img_size, img_size))
        x_train.append(list(image_array))
        y_train.append(code[i])

# Convert to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

100%|██████████| 52/52 [00:08<00:00,  6.34it/s]

(104, 220, 220, 3)
(104,)


In [9]:
x_test = []
y_test = []
img_size = 220
for i in os.listdir(path_test):
    files = gb.glob(pathname=str(os.path.join(path_test, i, '*.jpg')))

    for j in tqdm(files):
        image = cv2.imread(j)
        image_array = cv2.resize(image, (img_size, img_size))
        x_test.append(list(image_array))
        y_test.append(code[i])


# Convert to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

100%|██████████| 12/12 [00:02<00:00,  5.91it/s]

(24, 220, 220, 3)
(24,)


In [10]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
model = Sequential()

for layer in model_vgg16.layers:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False # Not trainable weights

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 220, 220, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 220, 220, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 110, 110, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 110, 110, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 110, 110, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 55, 55, 128)       0         
                                                         

In [11]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [12]:
early_stopping_monitor = EarlyStopping(patience = 2)
model.fit(x_train,y_train,epochs = 5,batch_size = 32,callbacks = [early_stopping_monitor])

Epoch 1/5
4/4 [==============================] - ETA: 0s - loss: 4.6455 - accuracy: 0.5865 

4/4 [==============================] - 65s 15s/step - loss: 4.6455 - accuracy: 0.5865
Epoch 2/5
4/4 [==============================] - ETA: 0s - loss: 1.2017 - accuracy: 0.8942 

4/4 [==============================] - 65s 15s/step - loss: 1.2017 - accuracy: 0.8942
Epoch 3/5
4/4 [==============================] - ETA: 0s - loss: 0.5048 - accuracy: 0.9615 

4/4 [==============================] - 62s 14s/step - loss: 0.5048 - accuracy: 0.9615
Epoch 4/5
4/4 [==============================] - ETA: 0s - loss: 0.1422 - accuracy: 0.9904 

4/4 [==============================] - 63s 15s/step - loss: 0.1422 - accuracy: 0.9904
Epoch 5/5
4/4 [==============================] - ETA: 0s - loss: 0.1133 - accuracy: 0.9808 

4/4 [==============================] - 65s 15s/step - loss: 0.1133 - accuracy: 0.9808


In [13]:
model.evaluate(x_test,y_test.astype('float32'))

1/1 [==============================] - 16s 16s/step - loss: 1.3787e-28 - accuracy: 1.0000


[1.3787025896319874e-28, 1.0]

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
